# Vorbereitung

Wenn noch nicht im ersten Teil ausgeführt, bitte zuerst folgende Bibliotheken installieren.

In [1]:
%pip install langchain openai python-dotenv tiktoken

Note: you may need to restart the kernel to use updated packages.


# Prompts und Prompt-Templates

Die Prompts (Eingaben) sind praktisch unsere einzige Möglichkeit mit der KI zu kommunizieren. Neben der eigentlichen Aufgabenstellung enthalten die Prompts ggfs. zusätzliche Informationen (Kontext). Das können Information, die dem Sprachmodell ermöglichen mit unseren Daten zu arbeiten ("In context learning"), die Chat-Historie uvm.
Das bedeutet, dass es notwendig ist den Prompt programmatisch aufzubauen. LangChain bietet hierfür die Prompt-Templates an.
Zuerst ein Beispiel ohne LangChain:

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessage, HumanMessage

system_prompt = "Du bist ein hilfreicher Assistent und hilfst mir beim Thema künstliche Intelligenz"
user_prompt = """
Bitte fasse die wichtigsten Punkte des Artikels als eine Strichliste zusammen:

Artikel:
Starke und schwache KI
	
Starke KI wären Computersysteme, die auf Augenhöhe mit Menschen die Arbeit zur Erledigung schwieriger Aufgaben übernehmen können. Demgegenüber geht es bei schwacher KI darum, konkrete Anwendungsprobleme zu meistern. Das menschliche Denken und technische Anwendungen sollen hier in Einzelbereichen unterstützt werden.[2] Die Fähigkeit zu lernen ist eine Hauptanforderung an KI-Systeme und muss ein integraler Bestandteil sein, der nicht erst nachträglich hinzugefügt werden darf. Ein zweites Hauptkriterium ist die Fähigkeit eines KI-Systems, mit Unsicherheit und probabilistischen Informationen umzugehen.[8] Insbesondere sind solche Anwendungen von Interesse, zu deren Lösung nach allgemeinem Verständnis eine Form von „Intelligenz“ notwendig zu sein scheint. Letztlich geht es der schwachen KI somit um die Simulation intelligenten Verhaltens mit Mitteln der Mathematik und der Informatik, es geht ihr nicht um Schaffung von Bewusstsein oder um ein tieferes Verständnis von Intelligenz. Während die Schaffung starker KI an ihrer philosophischen Fragestellung bis heute scheiterte, sind auf der Seite der schwachen KI in den letzten Jahren bedeutende Fortschritte erzielt worden.

Ein starkes KI-System muss nicht viele Gemeinsamkeiten mit dem Menschen haben. Es wird wahrscheinlich eine andersartige kognitive Architektur aufweisen und in seinen Entwicklungsstadien ebenfalls nicht mit den evolutionären kognitiven Stadien des menschlichen Denkens vergleichbar sein (Evolution des Denkens). Vor allem ist nicht anzunehmen, dass eine künstliche Intelligenz Gefühle wie Liebe, Hass, Angst oder Freude besitzt.[9]
"""

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt)
    ]

llm = ChatOpenAI()

llm(messages)

AIMessage(content='- Starke KI ist in der Lage, schwierige Aufgaben auf Augenhöhe mit Menschen zu erledigen, während schwache KI sich auf konkrete Anwendungsprobleme konzentriert.\n- Lernen und der Umgang mit Unsicherheit sind Hauptanforderungen an KI-Systeme.\n- Schwache KI simuliert intelligentes Verhalten mit mathematischen und informatischen Mitteln, ohne Bewusstsein oder ein tiefes Verständnis von Intelligenz zu schaffen.\n- Fortschritte wurden bei schwacher KI erzielt, während die Schaffung starker KI bisher an philosophischen Fragen gescheitert ist.\n- Starke KI muss nicht menschenähnlich sein und wird wahrscheinlich eine andere kognitive Architektur haben.\n- Künstliche Intelligenz besitzt keine menschenähnlichen Gefühle wie Liebe, Hass, Angst oder Freude.')

Das Beispiel zeigt, wie man "zu Fuß" den Kontext für die Aufgabenstellung aufbauen kann.

# Prompt-Templates Grundlagen

Schauen wir uns an, wie LangChain Prompt-Templates funktionieren

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Schlage mir zehn Namen für {ding} vor und formatiere die Ausgabe als {format}."
)

prompt_template.format(ding="Hunde", format="HTML")

'Schlage mir zehn Namen für Hunde vor und formatiere die Ausgabe als HTML.'

Das Beispiel zeigt die Grundfunktionalität eines basis Prompt-Templates. Es gibt eine Reihe von vordefinierten Prompt-Template-Klassen für unterschiedliche Anwendungsfälle. So gibt es z.B. das Template _FewShotChatMessagePromptTemplate_ das automatisch Beispiele für richtige und falsche Antworten in den Prompt einfügt, damit man das sog. _Few-Shot-Prompt_ aufbauen können. Darunter versteht man das Verfahren, bei dem man der KI hilft die Antwort korrekt aufzubauen, in dem man einige Beispiele für richtige Antworten nennt.

Schauen wir uns das Beispiel von oben an, diesmal allerdings mit Prompt-Template.

In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

artikel = """
Starke und schwache KI

Starke KI wären Computersysteme, die auf Augenhöhe mit Menschen die Arbeit zur Erledigung schwieriger Aufgaben übernehmen können. Demgegenüber geht es bei schwacher KI darum, konkrete Anwendungsprobleme zu meistern. Das menschliche Denken und technische Anwendungen sollen hier in Einzelbereichen unterstützt werden.[2] Die Fähigkeit zu lernen ist eine Hauptanforderung an KI-Systeme und muss ein integraler Bestandteil sein, der nicht erst nachträglich hinzugefügt werden darf. Ein zweites Hauptkriterium ist die Fähigkeit eines KI-Systems, mit Unsicherheit und probabilistischen Informationen umzugehen.[8] Insbesondere sind solche Anwendungen von Interesse, zu deren Lösung nach allgemeinem Verständnis eine Form von „Intelligenz“ notwendig zu sein scheint. Letztlich geht es der schwachen KI somit um die Simulation intelligenten Verhaltens mit Mitteln der Mathematik und der Informatik, es geht ihr nicht um Schaffung von Bewusstsein oder um ein tieferes Verständnis von Intelligenz. Während die Schaffung starker KI an ihrer philosophischen Fragestellung bis heute scheiterte, sind auf der Seite der schwachen KI in den letzten Jahren bedeutende Fortschritte erzielt worden.

Ein starkes KI-System muss nicht viele Gemeinsamkeiten mit dem Menschen haben. Es wird wahrscheinlich eine andersartige kognitive Architektur aufweisen und in seinen Entwicklungsstadien ebenfalls nicht mit den evolutionären kognitiven Stadien des menschlichen Denkens vergleichbar sein (Evolution des Denkens). Vor allem ist nicht anzunehmen, dass eine künstliche Intelligenz Gefühle wie Liebe, Hass, Angst oder Freude besitzt.[9]
"""

system_prompt = "Du bist ein hilfreicher Assistent und hilfst mir beim Thema {thema}"
user_prompt = """
Bitte fasse die wichtigsten Punkte des Artikels als eine Strichliste zusammen:

Artikel:
{artikel}
"""

template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(user_prompt)
    ])

llm = ChatOpenAI()
llm(template.format_messages(
    thema="Künstliche Intelligenz",
    artikel=artikel
))


AIMessage(content='- Unterscheidung zwischen starker und schwacher KI\n- Starke KI kann schwierige Aufgaben auf Augenhöhe mit Menschen erledigen, schwache KI meistert konkrete Anwendungsprobleme\n- Lernen und Umgang mit Unsicherheit sind Hauptanforderungen an KI-Systeme\n- Schwache KI simuliert intelligentes Verhalten mit mathematischen und informatischen Mitteln, nicht um Schaffung von Bewusstsein oder tiefes Verständnis von Intelligenz\n- Fortschritte bei schwacher KI, starke KI bisher gescheitert\n- Starke KI kann andersartige kognitive Architektur haben und besitzt keine Gefühle wie Liebe oder Hass')